In [7]:
import re

def clean_and_split_sentences(text):
    """
    Очищает текст и разбивает на предложения.
    Для корейского используем простую эвристику: .!?… (корейские знаки препинания).
    """
    # 1. Удаляем метаданные (от начала до "차례" или "제 1장")
    start_patterns = [r'차례', r'제 1장', r'제1장']
    for pattern in start_patterns:
        match = re.search(pattern, text)
        if match:
            text = text[match.start():]
            break

    # 2. Удаляем сноски типа '해리 포터'에 대한 찬사 외
    text = re.sub(r'[\'\"].*?외\.?\s*\d*', '', text)

    # 3. Разбиваем на предложения
    # Корейские завершающие знаки препинания: . ! ? … (многоточие)
    sentence_endings = r'[.!?…]+'
    sentences = re.split(sentence_endings, text)

    # 4. Фильтруем предложения
    valid_sentences = []
    for sent in sentences:
        sent = sent.strip()
        if not sent:
            continue

        # Удаляем номера глав (например: "제 11장 결투클럽.11")
        sent = re.sub(r'제\s*\d+\s*장\s*', '', sent)

        # Приблизительный подсчет слов (разбиваем по пробелам и корейским разделителям)
        # Для корейского лучше считать символы, но для вашей задачи подойдет
        words = re.findall(r'[\w]+', sent)  # Находит корейские и английские слова
        if 7 <= len(words) <= 70:
            # Проверяем, что предложение начинается с заглавной буквы
            if sent and sent[0].isupper():
                valid_sentences.append(sent + '.')
            else:
                # Если нет, делаем заглавной первую букву
                valid_sentences.append(sent[0].upper() + sent[1:] + '.')

    return valid_sentences

def process_book(file_path, output_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    sentences = clean_and_split_sentences(text)

    # Сохраняем нужное количество
    with open(output_path, 'w', encoding='utf-8') as f:
        for i, sent in enumerate(sentences[:]):
            f.write(f"{sent}\n")

    print(f"Сохранено {len(sentences)} предложений в {output_path}")
    return sentences

# Обработка двух книг
if __name__ == "__main__":
    # Обработка файлов
    book1_sentences = process_book("book_korean/processed_books/book_1_utf_8.txt", "book_korean/parsed_sentences/korean_sentences_1.txt")
    book2_sentences = process_book("book_korean/processed_books/book_2_utf_8.txt", "book_korean/parsed_sentences/korean_sentences_2.txt")
    book3_sentences = process_book("book_korean/processed_books/book_3_utf_8.txt", "book_korean/parsed_sentences/korean_sentences_3.txt")
    book4_sentences = process_book("book_korean/processed_books/book_4_utf_8.txt", "book_korean/parsed_sentences/korean_sentences_4.txt")

    # Объединяем, если нужно 12к из двух книг
    all_sentences = book1_sentences + book2_sentences + book3_sentences + book4_sentences
    with open(f"book_korean/parsed_sentences/korean_sentences_{len(all_sentences[:])}.txt", 'w', encoding='utf-8') as f:
        for sent in all_sentences[:]:
            f.write(f"{sent}\n")

    print(f"Итоговый файл: {len(all_sentences[:])} предложений")

Сохранено 3237 предложений в book_korean/parsed_sentences/korean_sentences_1.txt
Сохранено 3229 предложений в book_korean/parsed_sentences/korean_sentences_2.txt
Сохранено 5288 предложений в book_korean/parsed_sentences/korean_sentences_3.txt
Сохранено 8045 предложений в book_korean/parsed_sentences/korean_sentences_4.txt
Итоговый файл: 19799 предложений


In [10]:
import re
import os

def clean_sentence(sent):
    """
    Очистка предложения для TTS
    """
    if not sent or len(sent.strip()) == 0:
        return None

    # Убираем кавычки любого типа
    sent = re.sub(r'[\"\"\'\'「」『』《》]', '', sent)

    # Убираем скобки и их содержимое
    sent = re.sub(r'\([^)]*\)', '', sent)
    sent = re.sub(r'\[[^\]]*\]', '', sent)

    # Убираем специальные символы
    sent = re.sub(r'[#@$%&*_+=|~<>/\\]', '', sent)

    # Убираем римские цифры в начале (главы)
    sent = re.sub(r'^[IVXLCDM]+\.\s*', '', sent)

    # Убираем лишние пробелы
    sent = ' '.join(sent.split())

    # Удаляем точки в середине предложения (артефакты номеров страниц)
    sent = re.sub(r'(?<=\w)\.(?=\w)', '', sent)

    return sent.strip()

def clean_and_split_sentences(text):
    """
    Очищает текст и разбивает на предложения.
    """
    # Удаляем метаданные (от начала до "차례" или "제 1장")
    start_patterns = [r'차례', r'제 1장', r'제1장']
    for pattern in start_patterns:
        match = re.search(pattern, text)
        if match:
            text = text[match.start():]
            break

    # Удаляем сноски и примечания
    text = re.sub(r'[\'\"].*?외\.?\s*\d*', '', text)
    text = re.sub(r'옮긴이의 말.*', '', text, flags=re.DOTALL)

    # Заменяем многоточия на обычные точки для разделения
    text = text.replace('…', '.')

    # Разбиваем на предложения
    sentence_endings = r'[.!?]+'
    sentences = re.split(sentence_endings, text)

    # Фильтруем и очищаем предложения
    valid_sentences = []
    for sent in sentences:
        sent = clean_sentence(sent)
        if not sent:
            continue
            continue

        # Удаляем номера глав
        sent = re.sub(r'제\s*\d+\s*장\s*', '', sent)

        # Удаляем цифры (по требованию задания - цифры должны быть словами)
        # Лучше удалить предложения с цифрами, чем неправильно конвертировать
        if re.search(r'\d+', sent):
            continue

        # Минимальная длина - 7 символов (не слов!)
        if len(sent) < 7:
            continue

        # Проверяем что начинается с корейской или английской буквы
        first_char = sent[0]
        is_korean = ('가' <= first_char <= '힣')
        is_english = ('A' <= first_char <= 'Z') or ('a' <= first_char <= 'z')

        if not (is_korean or is_english):
            continue

        # Проверяем количество слов (разбиваем по пробелам)
        words = sent.split()
        if len(words) < 7:  # Минимум 7 слов по заданию
            continue

        # Максимум 70 слов по заданию
        if len(words) > 70:
            continue

        # Делаем первую букву заглавной если нужно
        if sent and sent[0].islower():
            sent = sent[0].upper() + sent[1:]

        # Добавляем точку в конце если её нет
        if not sent.endswith('.'):
            sent += '.'

        valid_sentences.append(sent)

    return valid_sentences

def process_all_books(book_folder, output_folder):
    """
    Обрабатывает все книги в папке и сохраняет результат
    """
    # Создаем папки если их нет
    os.makedirs(output_folder, exist_ok=True)

    all_sentences = []
    book_files = sorted([f for f in os.listdir(book_folder) if f.endswith('.txt')])

    print(f"Найдено {len(book_files)} книг для обработки")

    for book_file in book_files:
        book_path = os.path.join(book_folder, book_file)
        print(f"Обработка: {book_file}")

        try:
            with open(book_path, 'r', encoding='utf-8') as f:
                text = f.read()

            sentences = clean_and_split_sentences(text)
            all_sentences.extend(sentences)

            print(f"  Извлечено: {len(sentences)} предложений")
            print(f"  Всего накоплено: {len(all_sentences)} предложений")

        except Exception as e:
            print(f"  Ошибка при обработке {book_file}: {e}")

    # Сохраняем все отпарсенные предложения
    parsed_path = os.path.join(output_folder, "korean_parsed.txt")
    with open(parsed_path, 'w', encoding='utf-8') as f:
        for sent in all_sentences:
            f.write(f"{sent}\n")

    print(f"\nСохранен файл со всеми предложениями: {parsed_path}")
    print(f"Всего предложений: {len(all_sentences)}")

    # Сохраняем ровно 12000 предложений (если есть достаточно)
    final_path = os.path.join(output_folder, "korean_final_12000.txt")
    num_to_save = min(12000, len(all_sentences))

    with open(final_path, 'w', encoding='utf-8') as f:
        for i, sent in enumerate(all_sentences[:num_to_save]):
            f.write(f"{sent}\n")

    print(f"Сохранен финальный файл: {final_path}")
    print(f"Сохранено предложений: {num_to_save}")

    if len(all_sentences) < 12000:
        print(f"ВНИМАНИЕ: Недостаточно предложений! Нужно 12000, есть только {len(all_sentences)}")
        print("Добавьте больше книг или ослабьте критерии фильтрации.")

    return all_sentences

if __name__ == "__main__":
    # Укажите пути к вашим папкам
    book_folder = "book_korean/processed_books"
    output_folder = "book_korean/parsed_sentences"

    # Запускаем обработку
    all_sentences = process_all_books(book_folder, output_folder)

    # Выводим примеры для проверки
    print("\nПервые 5 предложений для проверки:")
    for i, sent in enumerate(all_sentences[:5]):
        print(f"{i+1}: {sent}")

    # Статистика по длине
    print("\nСтатистика по длине предложений:")
    lengths = [len(s.split()) for s in all_sentences[:1000]]  # Проверяем на первых 1000
    if lengths:
        print(f"  Среднее количество слов: {sum(lengths)/len(lengths):.1f}")
        print(f"  Минимальное количество слов: {min(lengths)}")
        print(f"  Максимальное количество слов: {max(lengths)}")
        print(f"  Предложений с более 70 слов: {len([l for l in lengths if l > 70])}")

Найдено 4 книг для обработки
Обработка: book_1_utf_8.txt
  Извлечено: 2841 предложений
  Всего накоплено: 2841 предложений
Обработка: book_2_utf_8.txt
  Извлечено: 2971 предложений
  Всего накоплено: 5812 предложений
Обработка: book_3_utf_8.txt
  Извлечено: 5049 предложений
  Всего накоплено: 10861 предложений
Обработка: book_4_utf_8.txt
  Извлечено: 7780 предложений
  Всего накоплено: 18641 предложений

Сохранен файл со всеми предложениями: book_korean/parsed_sentences\korean_parsed.txt
Всего предложений: 18641
Сохранен финальный файл: book_korean/parsed_sentences\korean_final_12000.txt
Сохранено предложений: 12000

Первые 5 предложений для проверки:
1: 차례차례 무게를 다는 걸 놀란 눈으로 바라보았다.
2: 해그리드가 마침내 쬐그마한 황금빛 열쇠 하나를 들어올리며 말했다.
3: 해그리드가 가슴에 손을 쭉 펴고, 거드름을 피며 말했다.
4: 해그리드는 일단 강아지용 비스킷을 다시 주머니에 쑤셔 넣고, 해리와 함께 그립훅을 따라 홀로 통하는 문 가운데 하나로 향했다.
5: 덤블도어 교수가 날 믿고 일을 맡긴 건데 네게 그걸 말하면 난 파면당할 거야.

Статистика по длине предложений:
  Среднее количество слов: 11.8
  Минимальное количество слов: 7
  Максимальное

In [13]:
import hashlib
from collections import Counter
import time

def check_duplicates(file_path, output_report_path=None):
    """
    Проверяет файл на дубликаты предложений
    Возвращает статистику и записывает отчет
    """
    print(f"\nПроверка файла: {file_path}")
    print("Чтение файла...")

    # Читаем все предложения
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = [line.strip() for line in f if line.strip()]

    total_lines = len(lines)
    print(f"Всего предложений: {total_lines}")

    # Используем хеши для быстрой проверки
    print("Вычисление хешей...")
    start_time = time.time()

    # Создаем хеши и считаем повторения
    hashes = []
    for line in lines:
        # Нормализуем: удаляем лишние пробелы, приводим к одному регистру
        normalized = ' '.join(line.split()).lower()
        line_hash = hashlib.md5(normalized.encode('utf-8')).hexdigest()
        hashes.append((line_hash, line))

    # Находим дубликаты
    hash_counter = Counter([h[0] for h in hashes])
    duplicate_hashes = {h: count for h, count in hash_counter.items() if count > 1}

    elapsed = time.time() - start_time
    print(f"Время проверки: {elapsed:.2f} секунд")

    # Собираем информацию о дубликатах
    duplicates_info = {}
    for line_hash, line_text in hashes:
        if line_hash in duplicate_hashes:
            if line_hash not in duplicates_info:
                duplicates_info[line_hash] = {
                    'count': hash_counter[line_hash],
                    'examples': [],
                    'first_index': None
                }
            if len(duplicates_info[line_hash]['examples']) < 3:  # Сохраняем до 3 примеров
                duplicates_info[line_hash]['examples'].append(line_text)

    # Находим индексы дубликатов
    print("Поиск позиций дубликатов...")
    for idx, (line_hash, line_text) in enumerate(hashes):
        if line_hash in duplicate_hashes:
            if duplicates_info[line_hash]['first_index'] is None:
                duplicates_info[line_hash]['first_index'] = idx + 1  # +1 для человеческого формата

    # Статистика
    num_unique = len(set([h[0] for h in hashes]))
    num_duplicates = total_lines - num_unique
    duplicate_lines = sum([count - 1 for count in hash_counter.values() if count > 1])

    print("\n" + "="*50)
    print("СТАТИСТИКА УНИКАЛЬНОСТИ:")
    print("="*50)
    print(f"Всего строк: {total_lines}")
    print(f"Уникальных строк: {num_unique}")
    print(f"Дублирующих строк: {num_duplicates}")
    print(f"Всего дубликатов (с повторениями): {duplicate_lines}")
    print(f"Процент уникальности: {(num_unique/total_lines*100):.2f}%")

    if duplicate_hashes:
        print(f"\nНайдено {len(duplicate_hashes)} уникальных предложений с дубликатами:")
        print("-"*50)

        for i, (h, info) in enumerate(list(duplicates_info.items())[:10]):  # Показываем первые 10
            print(f"\nДубликат #{i+1}:")
            print(f"  Количество повторений: {info['count']}")
            print(f"  Первое вхождение: строка {info['first_index']}")
            print(f"  Пример текста: \"{info['examples'][0][:80]}...\"")

        if len(duplicate_hashes) > 10:
            print(f"\n... и еще {len(duplicate_hashes) - 10} дубликатов")

    # Создаем отчет если указан путь
    if output_report_path:
        create_report(output_report_path, total_lines, num_unique, duplicates_info, lines, hashes)

    return {
        'total': total_lines,
        'unique': num_unique,
        'duplicate_count': len(duplicate_hashes),
        'duplicate_lines': duplicate_lines,
        'duplicates': duplicates_info
    }

def create_report(report_path, total, unique, duplicates_info, original_lines, hashes):
    """Создает подробный отчет о дубликатах"""
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write("="*60 + "\n")
        f.write("ОТЧЕТ О ПРОВЕРКЕ УНИКАЛЬНОСТИ\n")
        f.write("="*60 + "\n\n")

        f.write(f"Всего предложений: {total}\n")
        f.write(f"Уникальных предложений: {unique}\n")
        f.write(f"Дублирующих предложений: {total - unique}\n")
        f.write(f"Процент уникальности: {(unique/total*100):.2f}%\n\n")

        if duplicates_info:
            f.write("ПОДРОБНАЯ ИНФОРМАЦИЯ О ДУБЛИКАТАХ:\n")
            f.write("-"*60 + "\n\n")

            for i, (h, info) in enumerate(duplicates_info.items()):
                f.write(f"ДУБЛИКАТ #{i+1}\n")
                f.write(f"Количество повторений: {info['count']}\n")
                f.write(f"Первое вхождение: строка {info['first_index']}\n")
                f.write(f"Текст: {info['examples'][0]}\n")

                # Находим все позиции этого дубликата
                positions = []
                for idx, (line_hash, _) in enumerate(hashes):
                    if line_hash == h:
                        positions.append(idx + 1)

                f.write(f"Позиции в файле: {positions}\n")
                f.write("-"*40 + "\n\n")

            # Создаем список строк для удаления (оставляем только первое вхождение)
            f.write("РЕКОМЕНДАЦИИ ДЛЯ ОЧИСТКИ:\n")
            f.write("-"*60 + "\n")
            f.write("Строки для удаления (все, кроме первого вхождения):\n")

            lines_to_remove = []
            for h, info in duplicates_info.items():
                # Находим все позиции этого хеша
                positions = []
                for idx, (line_hash, _) in enumerate(hashes):
                    if line_hash == h:
                        positions.append(idx)

                # Оставляем первую позицию, остальные помечаем для удаления
                if len(positions) > 1:
                    lines_to_remove.extend(positions[1:])

            lines_to_remove.sort()
            for idx in lines_to_remove[:50]:  # Показываем первые 50
                f.write(f"Строка {idx + 1}: {original_lines[idx][:100]}...\n")

            if len(lines_to_remove) > 50:
                f.write(f"... и еще {len(lines_to_remove) - 50} строк\n")

            f.write(f"\nВсего строк для удаления: {len(lines_to_remove)}\n")

        else:
            f.write("ДУБЛИКАТЫ НЕ НАЙДЕНЫ. Файл полностью уникален.\n")

    print(f"\nПолный отчет сохранен в: {report_path}")

def remove_duplicates(input_path, output_path):
    """
    Создает новый файл без дубликатов, сохраняя порядок
    """
    print(f"\nСоздание файла без дубликатов...")

    seen_hashes = set()
    unique_lines = []
    duplicates_removed = 0

    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            text = line.strip()
            if not text:
                continue

            # Нормализуем и хешируем
            normalized = ' '.join(text.split()).lower()
            line_hash = hashlib.md5(normalized.encode('utf-8')).hexdigest()

            if line_hash not in seen_hashes:
                seen_hashes.add(line_hash)
                unique_lines.append(text)
            else:
                duplicates_removed += 1

    # Сохраняем уникальные строки
    with open(output_path, 'w', encoding='utf-8') as f:
        for line in unique_lines:
            f.write(f"{line}\n")

    print(f"Удалено дубликатов: {duplicates_removed}")
    print(f"Сохранено уникальных строк: {len(unique_lines)}")
    print(f"Новый файл: {output_path}")

    return len(unique_lines)

def compare_two_files(file1_path, file2_path):
    """
    Сравнивает два файла на общие предложения
    """
    print(f"\nСравнение файлов:")
    print(f"  Файл 1: {file1_path}")
    print(f"  Файл 2: {file2_path}")

    # Читаем оба файла
    with open(file1_path, 'r', encoding='utf-8') as f:
        lines1 = [line.strip() for line in f if line.strip()]

    with open(file2_path, 'r', encoding='utf-8') as f:
        lines2 = [line.strip() for line in f if line.strip()]

    print(f"  Предложений в файле 1: {len(lines1)}")
    print(f"  Предложений в файле 2: {len(lines2)}")

    # Создаем множества хешей
    hashes1 = set()
    for line in lines1:
        normalized = ' '.join(line.split()).lower()
        line_hash = hashlib.md5(normalized.encode('utf-8')).hexdigest()
        hashes1.add(line_hash)

    hashes2 = set()
    common_hashes = set()
    for line in lines2:
        normalized = ' '.join(line.split()).lower()
        line_hash = hashlib.md5(normalized.encode('utf-8')).hexdigest()
        hashes2.add(line_hash)
        if line_hash in hashes1:
            common_hashes.add(line_hash)

    # Статистика
    common_count = len(common_hashes)
    unique_to_file1 = len(hashes1) - common_count
    unique_to_file2 = len(hashes2) - common_count

    print("\nРЕЗУЛЬТАТЫ СРАВНЕНИЯ:")
    print(f"Общих предложений: {common_count}")
    print(f"Уникальных для файла 1: {unique_to_file1}")
    print(f"Уникальных для файла 2: {unique_to_file2}")

    if common_hashes:
        print(f"\nПримеры общих предложений:")
        # Находим примеры общих предложений
        common_examples = []
        for line in lines1:
            normalized = ' '.join(line.split()).lower()
            line_hash = hashlib.md5(normalized.encode('utf-8')).hexdigest()
            if line_hash in common_hashes and len(common_examples) < 5:
                common_examples.append(line)

        for i, example in enumerate(common_examples):
            print(f"{i+1}. {example[:80]}...")

    return common_count

if __name__ == "__main__":
    # Укажите пути к вашим файлам
    # parsed_file = "book_korean/parsed_sentences/korean_parsed.txt"
    final_file = "book_korean/parsed_sentences/korean_final_12000_unique.txt"

    # 1. Проверяем файл со всеми предложениями
    # stats1 = check_duplicates(
    #     parsed_file,
    #     "book_korean/parsed_sentences/duplicates_report_full.txt"
    # )

    # 2. Проверяем финальный файл (12000 предложений)
    stats2 = check_duplicates(
        final_file,
        "book_korean/parsed_sentences/duplicates_report_final.txt"
    )

    # 3. Сравниваем два файла
    # common = compare_two_files(parsed_file, final_file)
    #
    # # 4. Создаем очищенные версии если есть дубликаты
    # if stats1['duplicate_lines'] > 0:
    #     remove_duplicates(
    #         parsed_file,
    #         "book_korean/parsed_sentences/korean_parsed_unique.txt"
    #     )

    # if stats2['duplicate_lines'] > 0:
    #     remove_duplicates(
    #         final_file,
    #         "book_korean/parsed_sentences/korean_final_12000_unique.txt"
    #     )

    print("\n" + "="*60)
    print("ИТОГОВАЯ СТАТИСТИКА:")
    print("="*60)
    # print(f"Файл {parsed_file}:")
    # print(f"  Уникальность: {stats1['unique']}/{stats1['total']} ({(stats1['unique']/stats1['total']*100):.2f}%)")

    print(f"\nФайл {final_file}:")
    print(f"  Уникальность: {stats2['unique']}/{stats2['total']} ({(stats2['unique']/stats2['total']*100):.2f}%)")

    # if common > 0:
    #     print(f"\nВНИМАНИЕ: Найдено {common} общих предложений между файлами!")
    #     print("Это нормально, так как final_file является подмножеством parsed_file")


Проверка файла: book_korean/parsed_sentences/korean_final_12000_unique.txt
Чтение файла...
Всего предложений: 12000
Вычисление хешей...
Время проверки: 0.03 секунд
Поиск позиций дубликатов...

СТАТИСТИКА УНИКАЛЬНОСТИ:
Всего строк: 12000
Уникальных строк: 12000
Дублирующих строк: 0
Всего дубликатов (с повторениями): 0
Процент уникальности: 100.00%

Полный отчет сохранен в: book_korean/parsed_sentences/duplicates_report_final.txt

ИТОГОВАЯ СТАТИСТИКА:

Файл book_korean/parsed_sentences/korean_final_12000_unique.txt:
  Уникальность: 12000/12000 (100.00%)
